# Классификация новостей 
Цели: классифицировать новости по категориям исходя из их заголовка \
Датасет: https://www.kaggle.com/datasets/shivam271882/news-classification-dataset \
Датасет представляет из себя csv файл с столбцами headline, category, authors, short_description, отвечающие соответсвенно за заголовок, категорию, автора и короткое описание \
Чтобы не усложнять задачу будем использовать информацию только о заголовках, не трогая автора и короткое описание \
Размер датасета: 200853 семплов, но вследствии ограничения на вычислительные условия возмём только 50000 \
Метрики: accuracy, f1, roc_auc 

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import re
import string

from tqdm.notebook import tqdm 
from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer

import time
import optuna

import cudf
import cupy
from cuml.feature_extraction.text import TfidfVectorizer as cuTfidVectorizer
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.feature_extraction.text import CountVectorizer as cuCountVectorizer

from catboost import CatBoostClassifier, Pool

sns.set()

In [ ]:
seed = 42 # Для воспроизводимости 

In [ ]:
data = pd.read_csv('/kaggle/input/news-classification-dataset/News.csv').sample(50000, random_state=seed)
data.head()

# EDA

Размер классов

In [ ]:
labels, freqs = np.unique(data['category'], return_counts=True)

sorted_freqs_ix = np.argsort(freqs)[::-1]

In [ ]:
plt.figure(figsize=(20, 20))
sns.barplot(y=labels[sorted_freqs_ix], x=freqs[sorted_freqs_ix]);

Из графика понятно, что классы распределенны неравномерно и большинство новостей из тренировочного сета относятся к политике

Перед дальнейшим этапом анализа проведём небольшую предобработку текст

* приведём всё к нижнему регистру
* удалим все знаки препинания, цифры и нелатинские символы
* удалим все stopwords

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
    text = text.lower()  
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)    
    text = re.sub("([^\x00-\x7F])+", " ", text)
    return word_tokenize(text)


class ClearText(BaseEstimator, TransformerMixin):
    def fit(self, X=None, y=None):
        return self 
    
    def transform(self, X):
        res = list(map(clean_text, X))
        res = list(map(lambda sentence: list(filter(lambda word: word not in stop, sentence)), res))
        return res 

In [ ]:
clear_text = ClearText()
preprocessed_text = clear_text.fit_transform(data['headline'])

Теперь можно построить облако слов

In [ ]:
corpus = ''

for row in tqdm(preprocessed_text):
    corpus += ' '.join(row) + ' '

In [ ]:
wordcloud = WordCloud().generate(corpus)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Теперь разделим данные на тренировочную и тестовые выборки \
Будем делить в стандартных пропрциях sklearn, а именно train=0.7 и test = 0.3 \
Так же будем использовать стратифицированное разделение, чтобы и в трейне и в тесте пристуствовали примеры каждого класса \
Дополнительно перемешиваем данные

In [ ]:
train, test = train_test_split(data, stratify=data['category'], shuffle=True, random_state=seed)
train.shape, test.shape

In [ ]:
clear_text = ClearText()
train_processed = clear_text.fit_transform(train['headline'])
test_processed = clear_text.transform(test['headline'])

Теперь применим стеммизацию 

In [ ]:
snow_stemmer = SnowballStemmer(language='english')
stemmed_train= list(map(lambda x: ' '.join([snow_stemmer.stem(i) for i in x]), train_processed))
stemmed_test= list(map(lambda x: ' '.join([snow_stemmer.stem(i) for i in x]), test_processed))

Добавим столбец с предобработанным текстом в датасет и удалим ненужные столбцы

In [ ]:
train['tokens'] = stemmed_train
train.drop(columns=['headline', 'authors', 'short_description'], inplace=True)
test['tokens'] = stemmed_test
test.drop(columns=['headline', 'authors', 'short_description'], inplace=True)

Теперь закодируем категории новостей в числа: 0, 1, 2, 3, ... для обучения моделий 

In [ ]:
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['category'])
train.drop(columns=['category'], inplace=True)

test['label'] = label_encoder.transform(test['category'])
test.drop(columns=['category'], inplace=True)

Разобъем тренировочную выборки на две части (так же в соотношении 0.7:0.3)\
Большая часть будет так же называться тренировочной и использоваться для обучения моделей \
Меньшая для сравнения качества между моделями

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train['tokens'], train['label'], stratify=train['label'], random_state=seed)

In [ ]:
X_train.shape, X_val.shape

In [ ]:
model_selection_df = pd.DataFrame(columns=['clf', 'name', 'train_time', 'eval_time', 'accuracy','f1', 'roc_auc'])

In [ ]:
def get_scores(clf, X_val, y_true, f1_average='micro', row_auc_average='macro', multi_class='ovo', cuda=False):
    start = time.time()
    probas = clf.predict_proba(X_val)
    if cuda:
        probas = probas.get()
    end = time.time()
    
    y_pred = probas.argmax(1)
    return {
         'f1': f1_score(y_true, y_pred, average=f1_average),
         'accuracy': accuracy_score(y_true, y_pred),
         'roc_auc': roc_auc_score(y_true, probas, average=row_auc_average, multi_class=multi_class),
         'eval_time': end - start
    }

In [ ]:
def fit_evaluate_model(clf, name, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, cuda=False):
    res = {'name': name, 'clf': clf}
    
    print('Training started...')
    start = time.time()
    clf.fit(X_train, y_train);
    end = time.time()
    print(f'Training ended. Training time {end - start}s')

    res['train_time'] = end - start 
    
    print('\n')
    print('Evaluation started...')
    scores = get_scores(clf, X_val=X_val, y_true=y_val, cuda=cuda)
    res = {**res, **scores}
    print(f'Evaluation ended...')
    return res

Построим бейзлайн, попробуем метод близжайших соседей

In [ ]:
baseline = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('neighbors', KNeighborsClassifier())
])

res = fit_evaluate_model(baseline, 'count_vectorizer + neighbors')
model_selection_df = model_selection_df.append(res, ignore_index=True)

Попробуем подобрать гиперпараметры с помощью Grid Search

In [ ]:
vectorizerd_train = CountVectorizer().fit_transform(X_train)

params = {
    "n_neighbors": [2, 5, 10],
    "weights": ['uniform', 'distance']
}

knn_gs = GridSearchCV(KNeighborsClassifier(), params, cv=3, scoring='f1_micro')

knn_gs.fit(vectorizerd_train, y_train)

In [ ]:
knn_gs.best_params_

Обучим модель с наиболее эффективными параметрами и измерим качество

In [ ]:
knn_best = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('neighbors', KNeighborsClassifier(**knn_gs.best_params_))
])

res = fit_evaluate_model(knn_best, 'count_vectorizer + neighbors(best params)')
model_selection_df = model_selection_df.append(res, ignore_index=True)

Попробуем теперь Decision Tree

In [ ]:
tree = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('decision tree', DecisionTreeClassifier(random_state=seed))
])

res = fit_evaluate_model(tree, 'count_vectorizer + decision tree')
model_selection_df = model_selection_df.append(res, ignore_index=True)

Попробуем заменить Энкодер на более сложный 

In [ ]:
tree_tfidf = Pipeline([
    ('tf_idf', TfidfVectorizer()),
    ('decision tree', DecisionTreeClassifier(random_state=seed))
])

res = fit_evaluate_model(tree_tfidf, 'tf_idf + decision tree')
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
model_selection_df.sort_values('f1', ascending=False)

Попробуем логистическую регрессию

In [ ]:
log_reg_tfidf = Pipeline([
    ('tf_idf', TfidfVectorizer()),
    ('logistic regression', LogisticRegression(random_state=seed, solver='saga', max_iter=250))
])

res = fit_evaluate_model(log_reg_tfidf, 'tf_idf + logistic regression')
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
log_reg = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('logistic regression', LogisticRegression(random_state=seed, solver='saga', max_iter=250))
])

res = fit_evaluate_model(log_reg, 'count_vectorizer + logistic regression')
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
model_selection_df.sort_values('f1', ascending=False)

попробуем подобрать параметры с помощью Grid Search

In [ ]:
params = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 5],
}

log_reg_gs = GridSearchCV(LogisticRegression(solver='saga', max_iter=250), params, cv=3, scoring='f1_micro')

log_reg_gs.fit(vectorizerd_train, y_train)

In [ ]:
log_reg_gs.best_params_

In [ ]:
log_reg_best = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('logistic regression', LogisticRegression(random_state=seed, solver='saga', max_iter=250, **log_reg_gs.best_params_))
])

res = fit_evaluate_model(log_reg_best, 'count_vectorizer + logistic regression(best params)')
model_selection_df = model_selection_df.append(res, ignore_index=True)

Теперь попробуем обучить наивный баес \
Так как наивный баес не поддерживает работу с sparse матрица, сделаем дополнительный Estimator, для преобразования sparse матриц к dense

In [ ]:
class DenseTransformer(TransformerMixin):
    def __init__(self, cuda=False):
        self.cuda = cuda 
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        if self.cuda:
            return X.todense()
        return np.asarray(X.todense())

In [ ]:
naive_bias_tfidf = Pipeline([
    ('tf_idf', TfidfVectorizer()),
    ("dense_transformer", DenseTransformer()),
    ('naive_bias', GaussianNB())
])

res = fit_evaluate_model(naive_bias_tfidf, 'tf_idf + naive_bias')
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
naive_bias = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ("dense_transformer", DenseTransformer()),
    ('naive_bias', GaussianNB())
])

res = fit_evaluate_model(naive_bias, 'count_vectorizer + naive_bias')
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
model_selection_df[['name', 'f1', 'accuracy', 'roc_auc', 'train_time', 'eval_time']].sort_values('f1', ascending=False)

Теперь попробуем более сложные алгоритмы \
Вследствии, размера выборки и сложности алгоритмов, будем использовать cuml вместо sklearn, потому что данная библиотека позволяет обучать модели на GPU, что значительно ускорить работу \
Начнём с Random Forest

In [ ]:
rfc = Pipeline([
    ('count_vectorizer', cuCountVectorizer()),
    ('dense_transformer', DenseTransformer(cuda=True)),
    ('random_forest_classifier', cuRFC())
])

res = fit_evaluate_model(rfc, 'count_vectorizer + random_forest', cuda=True)
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
rfc_tfidf = Pipeline([
    ('tfidf', cuTfidVectorizer()),
    ('dense_transformer', DenseTransformer(cuda=True)),
    ('random_forest_classifier', cuRFC())
])

res = fit_evaluate_model(rfc, 'tfidf + random_forest', cuda=True)
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
model_selection_df[['name', 'f1', 'accuracy', 'roc_auc', 'train_time', 'eval_time']].sort_values('f1', ascending=False)

попробуем оптимизировать гиперпараметры с помощью optuna

In [ ]:
def objective(trial):
    params = {
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 10),
        "max_depth": trial.suggest_int("max_depth", 5, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
    }

    rfc = Pipeline([
        ('count_vectorizer', cuCountVectorizer()),
        ('dense_transformer', DenseTransformer(cuda=True)),
        ('random_forest_classifier', cuRFC(**params))
    ])
    
    rfc.fit(X_train, y_train)
    pred = rfc.predict(X_val)
    score = f1_score(y_val, pred.get(), average='micro')

    return score

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
best_rfc = Pipeline([
    ('count_vectorizer', cuCountVectorizer()),
    ('dense_transformer', DenseTransformer(cuda=True)),
    ('random_forest_classifier', cuRFC(**trial.params))
])

res = fit_evaluate_model(best_rfc, 'count_vectorizer + random_forest(best)', cuda=True)
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
model_selection_df[['name', 'f1', 'accuracy', 'roc_auc', 'train_time', 'eval_time']].sort_values('f1', ascending=False)

Теперь попробуем использовать градиентный бустинг. Для этого воспользуемся библиотекой catboost

In [ ]:
train_pool = Pool(pd.DataFrame(X_train), y_train, text_features=['tokens'])
val_pool = Pool(pd.DataFrame(X_val), y_val, text_features=['tokens'])

In [ ]:
model = CatBoostClassifier(iterations=2000,
                           early_stopping_rounds=50,
                           task_type="GPU",
                           loss_function='MultiClass',
                           devices='0:1')


train_time_start = time.time()
model.fit(train_pool, eval_set=val_pool)
train_time_end = time.time()

In [ ]:
val_time_start = time.time()
probas = model.predict_proba(val_pool)
preds = probas.argmax(-1)

scores = {'f1': f1_score(y_val, preds, average='micro'),
          'accuracy': accuracy_score(y_val, preds),
          'roc_auc': roc_auc_score(y_val, probas, multi_class='ovo')
         }
          
val_time_end = time.time()

In [ ]:
model_selection_df = model_selection_df.append({
    'clf': model,
    'name': 'gradient boosting',
    'train_time': train_time_end - train_time_start,
    'eval_time': val_time_end - val_time_start,
    **scores
}, ignore_index=True)

Последней используемой моделью, станет MLP. К сожалению, не удалось найти его в cuml, поэтому пришлось обучать на CPU с помощью sklearn

In [ ]:
mlp = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('mlp', MLPClassifier(early_stopping=True, max_iter=20))

])

res = fit_evaluate_model(mlp, 'count_vectorizer + mlp')
model_selection_df = model_selection_df.append(res, ignore_index=True)

In [ ]:
model_selection_df[['name', 'f1', 'accuracy', 'roc_auc', 'train_time', 'eval_time']].sort_values('f1', ascending=False)

Самая лучшая модель на текущий момент CounteVectorizer + MultiLayerPerceptron

Обучим модель на полной выборке и измерим качество на тесте

In [ ]:
final = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('mlp', MLPClassifier(early_stopping=True, max_iter=20))

])


final.fit(train['tokens'], train['label'])

In [ ]:
get_scores(clf=final, X_val=test['tokens'], y_true=test['label'])

В качестве демонстрации работы попробуем классифицировать следующую новость: 
**Top 10 tasty recipes**

делаем предобработку

In [ ]:
exp = ['Top 10 tasty recipes']

preprocessed_exp = clear_text.fit_transform(exp)
preprocessed_exp = list(map(lambda x: ' '.join([snow_stemmer.stem(i) for i in x]), preprocessed_exp))

Получаем индекс вероятной категории

In [ ]:
pred_exp_ix = final.predict(preprocessed_exp)
pred_exp_ix

Используем LabelEncoder для обратного преобразования индекса в категорию

In [ ]:
label_encoder.inverse_transform(pred_exp_ix)

Модель правильно классифицировала эту новость!

Ещё один пример \
Новость: How to be healthy

In [ ]:
exp = ['How to be healthy']

preprocessed_exp = clear_text.fit_transform(exp)
preprocessed_exp = list(map(lambda x: ' '.join([snow_stemmer.stem(i) for i in x]), preprocessed_exp))

pred_exp_ix = final.predict(preprocessed_exp)

label_encoder.inverse_transform(pred_exp_ix)

И это правильный ответ

последний пример

In [ ]:
exp = ['Biography of the best football player']

preprocessed_exp = clear_text.fit_transform(exp)
preprocessed_exp = list(map(lambda x: ' '.join([snow_stemmer.stem(i) for i in x]), preprocessed_exp))

pred_exp_ix = final.predict(preprocessed_exp)

label_encoder.inverse_transform(pred_exp_ix)

Выводы: 
* Наиболее эффективно себя показал MLP, показав f1 на 0.01 выше чем у близжайшего конкурента
* Можно сделать вывод, что даже простая комбинация CounterVectorizer + MLP может дать достаточно приличные результаты и правильно классифицировать новости
* Можно так же заметить, что не всегда более сложные алгоритмы дают лучшие результаты, так RandomForest(который считается довольно сложным алгоритмом, состоящим из множества маленьких) показал худшие результаты чем другие модели
* Можно заметить, что линейные модели показывают, наиболее высокие результаты в данной задаче, одной из причин для этого, может быть неспособность к экстраполяции у некоторых других алгоритмов

Пути развития: 
* Использовать не только заголовки, но и описание новости(и возможно даже автора)
* Обучать модели на всей выборке, а не только 50000 семплах
* Следует дальше пробовать нейронные сети, например RNN(GRU или LSTM), CNN, RNN with Attention, Transformers
* Попробовать использовать небольшие предобученные модели(roberta, distillbert, deberta)
* Попробовать связку эмбеддинги предобученной модели + какой-нибудь из алгоритмов выше(например градиентный бустинг)
* Попробовать подобрать гиперпараметры для градиентного бустинга и MLP с помощью optune
* Попробовать заняться feature engineering-ом, например, классифицировать каждый заголовок по интенту и добавить это как отдельную фичу
* Попробовать использовать LLM(например GPT3) и классифицировать с помощью подбора промптов
* Попробовать дообучить LLM с помощью одного из следующих методов: p-tuning, prefix-tuning, lora, adaptors 